# ✨ Overview of Functionality (Without UI)

**Description**: This Notebook demos how to run unbiased area estimation and outlines the main functionalities of the unbiased area estimation library. If you prefer to run the sampling design or analysis in a more interactive way, please use the run_sampling_design and run_analysis notebooks.

In [ ]:
%pip install -e .

In [ ]:
# Import all the required functions

from unbiased_area_estimation.config import Config
from unbiased_area_estimation.sampling_design import SamplingDesignPipeline

In [ ]:
# Load your config file

CONFIG_PATH = 'config.json' # Set the path to your config file here
config = Config.load_from_json(CONFIG_PATH)

In [ ]:
# Preprocess the raster and masks

sampling_design_pipeline = SamplingDesignPipeline(output_path=config.output_path, use_cached=config.use_cached, sampling_method=config.sampling.sampling_method)
regions = sampling_design_pipeline.preprocess(map_path=config.map_path, mask_paths=config.mask_paths, target_spatial_ref=config.target_spatial_ref, class_merge_map=config.class_merge_map)

In [ ]:
# Create the initial sample designs

sampling_designs, design_details = sampling_design_pipeline.create_and_save_sampling_designs(regions=regions, expected_uas=config.sampling.expected_uas, target_error=config.sampling.target_error, allocation_method_name=config.sampling.allocation_method)
print(sampling_designs)

In [ ]:
# OPTIONAL: Adapt sampling designs

# sampling_designs = {} # Copy the output from the cell above and modify according to your requirements.
target_errors = sampling_design_pipeline.get_expected_target_errors(region_names=[r.name for r in regions], sampling_designs=sampling_designs)
print(target_errors)

In [ ]:
# Create the actual sample sets

sample_sets = sampling_design_pipeline.sample_and_save(sampling_designs=sampling_designs, regions=regions, single_file=True)

In [ ]:
# You should now annotate your samples. Once you have done that, you can run the following code to estimate the area of each class in the raster.
# Use the original csv to save the annotated samples, if you do not want to change the paths.
# If running at a later point, ensure to load the config file again (Run the second and third code cells)
from unbiased_area_estimation.analysis import UnbiasedAreaEstimator

# Load the annotated samples
ANNOTATED_SAMPLES_FOLDER = config.output_path
ANNOTATED_COLUMN_NAME = 'Crop Type' # Change this to the column name in your annotated samples that contains the class labels

unbiased_area_estimator = UnbiasedAreaEstimator(results_dir=ANNOTATED_SAMPLES_FOLDER)
unbiased_area_estimates = unbiased_area_estimator.get_unbiased_area_estimates(annotated_column_name=ANNOTATED_COLUMN_NAME)
print(unbiased_area_estimates)